# Todo 
1. Load the dataset for 1 date, explore the different metrics (strike distribution, number of calls vs puts, expiry available, forward curve, etc)
2. For a given several maturities 𝑇_1,𝑇_2,𝑒𝑡𝑐, plot the implied volatility with respect to strike 𝐾, moneyness 𝑚, delta Δ for both calls and puts.
3. For a chosen maturity 𝑇_1, and using the associated 𝑆_𝑡,𝐹_𝑡,𝜎_(𝑚𝑘𝑡,𝑖) calibrate a SABR model. More precisely use Hagan’s formula (2002) and minimize the $SSE=\sum_{option\ i}\left(\sigma_{mkt,i}-\sigma_{SABR,i}\right)$ or MSE using scipy.minimize. You can use a L-BFGS-B solver.
     - Generalize to all the other smiles of this single day.
     - Using the tools provided compute the Black & Scholes (1973) greeks and prices, compare with initial values.
4. Repeat 3. but for an SSVI model Gatheral & Jacquier (2014) to get all the surface at once.
5. Generalize 3 or 4 to multiple days, compute the realized volatility for different windows.
6. Using the rule-based selection function explore the implied volatility structure through time and strikes, vs realized, skew etc.


In [ ]:
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc("font", **{"size": 22})

from investment_lab.option_selection import select_options
from investment_lab.metrics.volatility import rolling_realized_volatility
from investment_lab.metrics.util import levels_to_returns
from investment_lab.data.option_db import OptionLoader, extract_spot_from_options
from investment_lab.data.rates_db import USRatesLoader
from investment_lab.pricing.black_scholes import black_scholes_price, black_scholes_greeks
from investment_lab.rates import compute_forward
from investment_lab.metrics.distance import mse


In [ ]:
df_options = OptionLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4), process_kwargs={"ticker":"SPY"},)
df_options.head()

In [ ]:
df_spot = extract_spot_from_options(df_options)
df_spot.head()

In [ ]:
df_rates = USRatesLoader.load_data(datetime(2022, 1, 4), datetime(2022, 1, 4))
df_rates.head()

In [ ]:
df_options = compute_forward(df_options=df_options, df_rates=df_rates)
df_options.head()